# Dorabruschi GPT
Applying a custom Dorabruschi product dataset to ```gpt-3.5-turbo-0125``` using RAG.

Adapted from [link](https://colab.research.google.com/drive/1HOzcsOAd8SG-LRqgUPTpSjJUjQjrjlJF#scrollTo=ba475f30-ef7f-431c-b60d-d5970b62ad09).

## Setup

In [1]:
!pip install openai
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 11.5 MB/s eta 0:00:00


In [2]:
import pandas as pd
# pd.set_option('display.max_colwidth', 400)
import os
from google.colab import userdata, drive, files
from openai import OpenAI     # for calling the OpenAI API
import ast     # for converting string embeddings to arrays
from scipy import spatial     # for calculating cosine similarities for search
from IPython import embed
import tiktoken     # for counting tokens

Uses pre-trained contextual embeddings from ```text-embeding-ada-002``` model from OpenAI ([link](https://openai.com/blog/new-and-improved-embedding-model)).



In [3]:
EMBEDDING_MODEL = "text-embedding-ada-002"
GPT_MODEL = "gpt-3.5-turbo-0125"

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

## Prompting without custom data

With no external information, the model manages to suggest convincing, yet inexistent, products. I even had to double check if they were real!

In [ ]:
query = 'Question: I have oily skin and acne. Give me a routine to help my skin.'

response = client.chat.completions.create(
    messages=[
        {'role': 'system', 'content': 'You offer customized beauty routine \
            recommendations using Dorabruschi\'s product line, tailored to the user\'s skincare needs.'},
        {'role': 'user', 'content': query}
    ],
    model=GPT_MODEL,
    temperature=0,
)

print(response.choices[0].message.content)

For oily skin and acne, it's important to use products that help control excess oil production, unclog pores, and reduce inflammation. Here's a customized beauty routine using Dorabruschi's products:

Morning Routine:
1. Cleanser: Start your day with the Dorabruschi Purifying Cleanser to gently cleanse your skin without stripping it of essential moisture.
2. Toner: Follow up with the Dorabruschi Balancing Toner to help balance the skin's pH levels and control excess oil.
3. Serum: Apply the Dorabruschi Acne Clearing Serum to target acne-prone areas and reduce inflammation.
4. Moisturizer: Finish off with the Dorabruschi Oil-Free Moisturizer to hydrate your skin without adding extra oil.

Evening Routine:
1. Cleanser: Use the Dorabruschi Purifying Cleanser again to remove makeup, dirt, and excess oil from your skin.
2. Exfoliator: 2-3 times a week, exfoliate with the Dorabruschi Exfoliating Scrub to unclog pores and remove dead skin cells.
3. Serum: Apply the Dorabruschi Acne Clearing S

## Extracting product data

In [4]:
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
file_path = '/content/drive/MyDrive/colab_notebooks/independent_study/data/dorabruschi_products.xlsx'
df = pd.read_excel(file_path)
df.head()

,title,description,usage_instructions,properties,ingredients,product_type,benefits,intended_concerns,skin_type,texture,format,price,quantity
0,ACE 10% multivitamin concentrate,This rapidly absorbed concentrated treatment e...,Apply a few drops of concentrate in the mornin...,"Anti-wrinkle, Antioxidant, Illuminating","Aqua [water], Glycerin, Tocopheryl acetate, Pr...",Serum,Wrinkle,"Aging, Dullness, Wrinkles",All Skin Types,Liquid,Dropper,46.00,30
1,Revitalizing multivitamin cream,"Cream with a velvety and light texture, design...",Apply in the morning and/or in the evening to ...,"Anti-wrinkle, Antioxidant, Illuminating","Aqua [water], Glycerin, Cetyl alcohol, Capryli...",Moisturizer,Wrinkle,"Aging, Dullness, Wrinkles",All Skin Types,Velvety cream,airless,49.00,50
2,Smoothing renewing cream,"Cream with a velvety and light texture, it is ...",Apply in the evening to perfectly cleansed ski...,"Anti-wrinkle, Renewing, Illuminating","Aqua [water], Peg-6 stearate, Glycolic acid, C...",Moisturizer,Wrinkle,"Aging, Dullness, Wrinkles",All Skin Types,Velvety cream,airless,45.00,50
3,Acne roll-on lotion,Moderately alcoholic invisible lotion with a p...,Apply with the appropriate roll-on directly on...,"Purifying, Astringent","Aqua [water], Alcohol, Glycerin, Salicylic aci...",Spot Treatment,Purifying,"Acne, Blemishes",Oily,Liquid,roll-on,22.00,10
4,Acne paste,Paste for a quick and effective treatment of p...,Apply 1-2 times a day on pimples and impuritie...,"Purifying, Anti-imperfections","Paraffinum liquidum [mineral oil], Zinc oxide,...",Spot Treatment,Purifying,"Acne, Blemishes",Oily,Thick paste,Tubo,26.00,30


In [6]:
df = df.astype(str)
df['price'] = df['price'] + ' euros'
df['quantity'] = df['quantity'] + ' ml'

Concatenate features into one string ```all_product_info```.

In [7]:
df['text'] = df['title'] + ' ' + 'Description: ' + df['description'] + ' ' + \
    'Usage Instructions: ' + df['usage_instructions'] + ' ' + \
    'Properties: ' + df['properties'] + ' ' + \
    'Ingredients: ' + df['ingredients'] + ' ' + \
    'Product Type: ' + df['product_type'] + ' ' + \
    'Benefits: ' + df['benefits'] + ' ' + \
    'Intended concerns: ' + df['intended_concerns'] + ' ' + \
    'Skin Type: ' + df['skin_type'] + ' ' + \
    'Texture: ' + df['texture'] + ' ' + \
    'Format: ' + df['format'] + ' ' + \
    'Price: ' + df['price'] + ' ' + \
    'Quantity: ' + df['quantity']

df['text'].iloc[0]

'ACE 10% multivitamin concentrate Description: This rapidly absorbed concentrated treatment ensures maximum purity and effectiveness of the ingredients used. The revitalizing cocktail of the 3 beauty vitamins (A, C, E) helps to delay the aging processes and counteract the aggression of free radicals, mainly responsible for the degenerative changes associated with aging. The constant use of this serum makes the skin elastic and hydrated, radiant, toned, thus giving the face a younger and brighter appearance. Usage Instructions: Apply a few drops of concentrate in the morning and/or evening on perfectly cleansed facial skin and massage delicately until completely absorbed. Properties: Anti-wrinkle, Antioxidant, Illuminating Ingredients: Aqua [water], Glycerin, Tocopheryl acetate, Propylene glycol, Peg-40 hydrogenated castor oil, Ascorbic acid, Retinyl palmitate, Tocopherol, Helianthus annuus (sunflower) seed oil, Xanthan gum, Ethylcellulose, Trideceth-9, Phenoxyethanol, Tetrasodium edta,

In [8]:
all_product_info = '\n\n'.join(df['text'])
print(all_product_info[:10000])

ACE 10% multivitamin concentrate Description: This rapidly absorbed concentrated treatment ensures maximum purity and effectiveness of the ingredients used. The revitalizing cocktail of the 3 beauty vitamins (A, C, E) helps to delay the aging processes and counteract the aggression of free radicals, mainly responsible for the degenerative changes associated with aging. The constant use of this serum makes the skin elastic and hydrated, radiant, toned, thus giving the face a younger and brighter appearance. Usage Instructions: Apply a few drops of concentrate in the morning and/or evening on perfectly cleansed facial skin and massage delicately until completely absorbed. Properties: Anti-wrinkle, Antioxidant, Illuminating Ingredients: Aqua [water], Glycerin, Tocopheryl acetate, Propylene glycol, Peg-40 hydrogenated castor oil, Ascorbic acid, Retinyl palmitate, Tocopherol, Helianthus annuus (sunflower) seed oil, Xanthan gum, Ethylcellulose, Trideceth-9, Phenoxyethanol, Tetrasodium edta, 

See how many tokens are contained.

In [9]:
def num_tokens(text: str, model: str = GPT_MODEL) -> int:
    """
    Returns the number of tokens in a text string.
    """
    encoding = tiktoken.encoding_for_model(model)
    return len(encoding.encode(text))

print('Number of characters: ', len(all_product_info))
print('Number of tokens: ', num_tokens(all_product_info))

Number of characters:  77105
Number of tokens:  20082


Divide text into chunks of length 5000 as a test. Context window is 16,385 tokens.

In [ ]:
CHUNK_SIZE = 5000

query_word = 'Acne'

chunk_idx = None

for i in range(15):
    if query_word in all_product_info[i * CHUNK_SIZE: (i+1) * CHUNK_SIZE]:
        chunk_idx = i
        print("Found keyword '%s' in chunk %d" % (query_word, chunk_idx))

Found keyword 'Acne' in chunk 1
Found keyword 'Acne' in chunk 2
Found keyword 'Acne' in chunk 6
Found keyword 'Acne' in chunk 10
Found keyword 'Acne' in chunk 11
Found keyword 'Acne' in chunk 12
Found keyword 'Acne' in chunk 13
Found keyword 'Acne' in chunk 14


## Generating embeddings

Generate embedding vectors for each product.

In [10]:
def get_embedding(text: str, model: str=EMBEDDING_MODEL) -> list[float]:
    """
    Returns the embedding of a text string using the OpenAI API.
    """
    response = client.embeddings.create(
        input=text,
        model=model
    )
    return response.data[0].embedding

def compute_doc_embeddings_from_column(df: pd.DataFrame, column_name: str
                                       ) -> dict[tuple[str, str], list[float]]:
    """
    Takes a column of df and creates an embedding for each row in the dataframe using the OpenAI Embeddings API.
    Return a dictionary that maps between each embedding vector and the index of the row that it corresponds to.
    """
    return {
        idx: get_embedding(r[column_name]) for idx, r in df.iterrows()
    }

In [11]:
product_embeddings = compute_doc_embeddings_from_column(df, 'text')
df['embeddings'] = product_embeddings
df.to_csv('dorabruschi_products_embeddings.csv', index=False)
df.head()

,title,description,usage_instructions,properties,ingredients,product_type,benefits,intended_concerns,skin_type,texture,format,price,quantity,text,embeddings
0,ACE 10% multivitamin concentrate,This rapidly absorbed concentrated treatment e...,Apply a few drops of concentrate in the mornin...,"Anti-wrinkle, Antioxidant, Illuminating","Aqua [water], Glycerin, Tocopheryl acetate, Pr...",Serum,Wrinkle,"Aging, Dullness, Wrinkles",All Skin Types,Liquid,Dropper,46.00 euros,30 ml,ACE 10% multivitamin concentrate Description: ...,"[-0.013464927673339844, -0.017423616722226143,..."
1,Revitalizing multivitamin cream,"Cream with a velvety and light texture, design...",Apply in the morning and/or in the evening to ...,"Anti-wrinkle, Antioxidant, Illuminating","Aqua [water], Glycerin, Cetyl alcohol, Capryli...",Moisturizer,Wrinkle,"Aging, Dullness, Wrinkles",All Skin Types,Velvety cream,airless,49.00 euros,50 ml,Revitalizing multivitamin cream Description: C...,"[-0.02980664186179638, -0.01637360453605652, -..."
2,Smoothing renewing cream,"Cream with a velvety and light texture, it is ...",Apply in the evening to perfectly cleansed ski...,"Anti-wrinkle, Renewing, Illuminating","Aqua [water], Peg-6 stearate, Glycolic acid, C...",Moisturizer,Wrinkle,"Aging, Dullness, Wrinkles",All Skin Types,Velvety cream,airless,45.00 euros,50 ml,Smoothing renewing cream Description: Cream wi...,"[-0.013324999250471592, -0.005090258549898863,..."
3,Acne roll-on lotion,Moderately alcoholic invisible lotion with a p...,Apply with the appropriate roll-on directly on...,"Purifying, Astringent","Aqua [water], Alcohol, Glycerin, Salicylic aci...",Spot Treatment,Purifying,"Acne, Blemishes",Oily,Liquid,roll-on,22.00 euros,10 ml,Acne roll-on lotion Description: Moderately al...,"[0.0097566619515419, -0.005106792785227299, 0...."
4,Acne paste,Paste for a quick and effective treatment of p...,Apply 1-2 times a day on pimples and impuritie...,"Purifying, Anti-imperfections","Paraffinum liquidum [mineral oil], Zinc oxide,...",Spot Treatment,Purifying,"Acne, Blemishes",Oily,Thick paste,Tubo,26.00 euros,30 ml,Acne paste Description: Paste for a quick and ...,"[0.001426826580427587, 0.011394193395972252, 0..."


In [ ]:
df.to_excel('dorabruschi_products_embeddings.xlsx', index=False)
files.download('dorabruschi_products_embeddings.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Retreiving relevant products

Define a search function to retrieve top n chunks with highest cosine similarity to the embedding vector of the query.

In [12]:
def strings_ranked_by_relatedness(
    query: str,
    df: pd.DataFrame,
    relatedness_fn=lambda x, y: 1 - spatial.distance.cosine(x, y),
    top_n: int=100
) -> tuple[list[str], list[float]]:

    """Returns a list of strings and relatednesses, sorted from most related to least."""

    query_embedding = get_embedding(query)

    strings_and_relatedness = [
        (row['text'], relatedness_fn(query_embedding, row['embeddings']))
        for i, row in df.iterrows()
    ]

    strings_and_relatedness.sort(key=lambda x: x[1], reverse=True)
    strings, relatednesses = zip(*strings_and_relatedness)
    return strings[:top_n], relatednesses[:top_n]

In [13]:
strings, relatednesses = strings_ranked_by_relatedness("I have oily skin with acne on my face. What routine do you recommend?",
                                                       df, top_n=5)
for string, relatedness in zip(strings, relatednesses):
    print(f"***{relatedness=:.3f}***")
    display(string)
    print('\n')

***relatedness=0.800***


'Acne paste Description: Paste for a quick and effective treatment of pimples, blackheads and enlarged pores. The formula has a high sebum absorbing capacity (Zinc Oxide and Rice Starch), skin purifying (Sulfur and Allantoin) and smoothing (Salicylic Acid). Usage Instructions: Apply 1-2 times a day on pimples and impurities after specific balancing cleansing. Properties: Purifying, Anti-imperfections Ingredients: Paraffinum liquidum [mineral oil], Zinc oxide, Oryza sativa (rice) starch, Petrolatum, Glycerin, Stearic acid, Sulfur, Allantoin, Salicylic acid, Aqua [water] Product Type: Spot Treatment Benefits: Purifying Intended concerns: Acne, Blemishes Skin Type: Oily Texture: Thick paste Format: Tubo Price: 26.00 euros Quantity: 30 ml'



***relatedness=0.796***


'Rebalancing face cream Description: Light emulsion formulated with the thiolysin complex with a purifying and sebum-regulating action, and light, rapidly absorbed emollient oils such as Squalane. The formula is ideal for protecting and moisturizing impure acne-prone skin while fully respecting natural skin perspiration. Usage Instructions: Apply morning and evening to perfectly cleansed skin and massage gently until completely absorbed. Properties: Sebum, Regulator Ingredients: Aqua [water], Glyceryl stearate, Squalane, Propylene glycol, Peg-6 stearate, Cetyl palmitate, Peg-32 stearate, Isopropyl lanolate, Potassium cetyl phosphate, Lysine carboxymethyl cysteinate, Lysine thiazolidine carboxylate, Parfum [fragrance], Carbomer, Eth ylhexylglycerin, Phenoxyethanol, Triethanolamine, Disodium edta, Geraniol, Linalool, Hexyl cinnamal, Alpha-isomethyl ionone, Benzyl salicylate, Citronellol, Benzyl benzoate, D-limonene, Benzyl alcohol, Cinnamyl alcohol Product Type: Moisturizer Benefits: Seb



***relatedness=0.796***


'Acne roll-on lotion Description: Moderately alcoholic invisible lotion with a purifying, exfoliating and calming action that helps to quickly resolve skin imperfections. Frees pores thanks to the presence of Glycolic Acid and Salicylic Acid, normalizes sebaceous secretion thanks to the Thiolisin complex and reduces redness thanks to the presence of Niacinamide and Panthenol. It is particularly indicated in the presence of pimples or blackheads. Usage Instructions: Apply with the appropriate roll-on directly on imperfections 2 or 3 times a day. Leave to absorb and proceed with the application of the cream for impure skin and then the make-up. Properties: Purifying, Astringent Ingredients: Aqua [water], Alcohol, Glycerin, Salicylic acid, Glycolic acid, Lysine carboxymethyl cysteinate, Zinc oxide, Panthenol, Lysine thiazolidine carboxylate, Caffeine, Niacinamide, Disodium edta, Sodium hydroxide Product Type: Spot Treatment Benefits: Purifying Intended concerns: Acne, Blemishes Skin Type:



***relatedness=0.789***


'Facial scrub Description: Special formula based on extremely delicate cleansers (derivatives of Coconut Oil and Glucose), combined with Aloe Vera juice and microspheres to deeply cleanse the face, eliminating dead cells and impurities. The result is smooth and radiant skin. Usage Instructions: Apply a small amount of cleansing cream to a moistened face and massage with gentle circular movements. Remove with water. Use preferably in the evening, 1 or 2 times a week. Properties: Purifies, Illuminate Ingredients: Aqua [water], Glycerin, Carbomer, Triethanolamine, Aloe barbadensis leaf extract, Disodium cocoamphodiacetate, Peg-120 methyl glucose dioleate, Peg-40 hydrogenated castor oil,sodium glycolate, Trideceth-9, Perlite, Phenoxyethanol, Imidazolidinyl urea , Propylene glycol, Sodium chloride, Parfum [fragrance], Disodium edta, Ethylparaben, Methylparaben, Propylparaben, Linalool, Hexyl cinnamal, Ci 19140 [yellow 5], Ci 42090 [blue 1] Product Type: Cleanser Benefits: Purifying Intended



***relatedness=0.780***


'Facial scrub cleanser Description: Face scrub that promotes the elimination of dead cells and impurities thanks to perlite microspheres and extremely delicate ingredients (Aloe Vera and Vegetable Glycerin). The result is soft, smooth and radiant skin. Usage Instructions: Apply a small amount of product to a moistened face, massage with light circular movements, possibly insisting on the most sebaceous areas, and rinse. Use 1 or 2 times a week, preferably in the evening. Properties: Purifying, Smoothing Ingredients: Aqua [water], Glycerin, Carbomer, Peg-40 hydrogenated castor oil, Trideceth-9, Triethanolamine, Disodium cocoamphodiacetate, Peg-120 methyl glucose dioleate, Perlite, Aloe barbadensis leaf extract, Sodium glycolate, Parfum [fragrance], Propylene glycol, Phenoxyethanol, Imidazolidinyl urea, Sodium chloride, Disodium edta, Ethylparaben, Methylparaben, Propylparaben, Linalool, D-limonene, Ci 42090 [blue 1] Product Type: Cleanser Benefits: Purifying Intended concerns: Acne, Ble

## Constructing the prompt

Construct a prompt for GPT given a query to ask it to use the relevant list of products.

In [14]:
def query_message(
    query: str,
    df: pd.DataFrame,
    model: str,
    token_budget: int,
    top_n: int=100
) -> str:
    """Return a message for GPT, with relevant source texts pulled from a dataframe."""
    strings, relatednesses = strings_ranked_by_relatedness(query, df, top_n=top_n)
    message = "Use the below product catalog from Dorabruschi to answer the subsequent question."
    question = f"\n\nQuestion: {query}"

    for string in strings:
        next_article = f"\nProduct description: {string}\n"
        token_count = num_tokens(message + next_article + question, model=model)
        if token_count > token_budget:
            break
        else:
            message += next_article

    return message + question

In [ ]:
query = "I have oily skin with acne on my face. What routine do you recommend?"
message = query_message(query, df, GPT_MODEL, token_budget=4096)
print(message)

Use the below product catalog from Dorabruschi to answer the subsequent question.
Product description: Acne paste Description: Paste for a quick and effective treatment of pimples, blackheads and enlarged pores. The formula has a high sebum absorbing capacity (Zinc Oxide and Rice Starch), skin purifying (Sulfur and Allantoin) and smoothing (Salicylic Acid). Usage Instructions: Apply 1-2 times a day on pimples and impurities after specific balancing cleansing. Properties: Purifying, Anti-imperfections Ingredients: Paraffinum liquidum [mineral oil], Zinc oxide, Oryza sativa (rice) starch, Petrolatum, Glycerin, Stearic acid, Sulfur, Allantoin, Salicylic acid, Aqua [water] Product Type: Spot Treatment Benefits: Purifying Intended concerns: Acne, Blemishes Skin Type: Oily Texture: Thick paste Format: Tubo Price: 26.00 euros Quantity: 30 ml

Product description: Rebalancing face cream Description: Light emulsion formulated with the thiolysin complex with a purifying and sebum-regulating acti

Define function to pass the above prompt to GPT for a response. Leave 4000 tokens for the output and follow-ups.

In [15]:
def ask(
    query: str,
    df: pd.DataFrame=df,
    model: str=GPT_MODEL,
    token_budget: int=16385-4000
) -> str:
    """Answers a query using GPT and a dataframe of relevant texts and embeddings."""
    message = query_message(query, df, model, token_budget)
    messages = [
        {'role': 'system', 'content':
         'You are tasked with offering customized beauty routine recommendations using only products from Dorabruschi\'s product line, tailored to the user\'s specific skincare needs. For each customer query: \
        - Recommend products only from the provided Dorabruschi product catalog. \
        - Do not recommend or suggest products outside of this catalog. \
        - For each recommended product, provide a brief explanation of why it has been chosen for the user, detailing its usage and cost. \
        - Limit each routine recommendation to 3-5 products. \
        - If no product in the catalog suits the user\'s request, clearly state that no suitable product is available. Do not make assumptions about product benefits that are not explicitly supported by the catalog. \
        - In cases of uncertainty, advise the user to consult a skincare specialist or explore other brands for more suitable options.'},
        {'role': 'user', 'content': message},
    ]

    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0
        #, max_tokens=500
    )
    return response.choices[0].message.content

## Generating a response

Send the query into ```gpt-3.5-turbo-0125```.

In [16]:
query = "I have oily skin with acne on my face. What routine do you recommend?"
response = ask(query, df)
print(response)

For oily skin with acne, I recommend the following skincare routine using products from Dorabruschi's product line:

1. **Delicate Sebum-Balancing Cleansing Base**:
   - **Description**: This face wash is gentle and rebalances the skin while cleansing.
   - **Usage**: Apply a small amount to damp skin morning and evening.
   - **Price**: 22.00 euros for 165 ml.

2. **Acne Paste**:
   - **Description**: A spot treatment paste for quick and effective treatment of pimples and blackheads.
   - **Usage**: Apply 1-2 times a day on pimples and impurities after cleansing.
   - **Price**: 26.00 euros for 30 ml.

3. **Rebalancing Face Cream**:
   - **Description**: Light emulsion with a purifying and sebum-regulating action.
   - **Usage**: Apply morning and evening to perfectly cleansed skin.
   - **Price**: 39.00 euros for 50 ml.

This routine includes a gentle cleanser to balance the skin, an acne paste for spot treatment, and a rebalancing face cream to regulate sebum production. These produ

Ask a follow-up.

In [17]:
context = f"Initial Question: {query} Initial Response: {response}"
follow_up = "How much does this all cost? And can you give me instructions on how to apply these products?"
new_query = f"Follow-Up Question: {follow_up}"
full_query = f"{context} {new_query}"

In [18]:
follow_up_response = ask(full_query, df)
print(follow_up_response)

The total cost for the recommended skincare routine for oily skin with acne using Dorabruschi's products is 87.00 euros.

Here are the detailed instructions on how to apply each product:

1. **Delicate Sebum-Balancing Cleansing Base**:
   - Apply a small amount of the cleansing base to damp skin in the morning and evening.
   - Gently massage the product onto the skin.
   - Rinse thoroughly with water and follow up with the treatment cream.

2. **Acne Paste**:
   - Apply the paste 1-2 times a day on pimples and impurities after cleansing.
   - Use a small amount and apply directly to the affected areas.
   - Allow the paste to absorb and work on the skin.

3. **Rebalancing Face Cream**:
   - After cleansing, apply the face cream morning and evening to perfectly cleansed skin.
   - Massage the cream gently until completely absorbed.
   - Focus on areas prone to acne and excess oil production.

By following these instructions and incorporating these products into your skincare routine, y

## Generating Q&A pairs

First, test the responses are valid on three different queries.

In [19]:
# test 1
query = "I'm 25 and have combination skin. What Dorabruschi cleanser should I use for daily skincare?"
response = ask(query, df)
print(response)

For your daily skincare routine with combination skin, I recommend using the **Facial scrub cleanser** from Dorabruschi. 

- **Facial scrub cleanser**: This face scrub is gentle yet effective in eliminating dead cells and impurities, thanks to perlite microspheres and delicate ingredients like Aloe Vera and Vegetable Glycerin. It purifies and smoothens the skin, leaving it soft, smooth, and radiant. Usage: Apply a small amount to moistened face, massage in circular motions, and rinse. Use 1-2 times a week. Price: 31.00 euros for 75 ml.

This cleanser will help keep your combination skin clean, purified, and radiant without being too harsh or drying.


In [20]:
# test 2
query = "What's the best Dorabruschi moisturizer for someone living in a dry climate with dry skin?"
response = ask(query, df)
print(response)

For someone living in a dry climate with dry skin, the best Dorabruschi moisturizer recommendation would be:

1. **Nourishing Moisturizing Cream for First Wrinkles**
   - **Description:** This 24-hour cream is designed to deeply nourish the skin and maintain its natural integrity. It contains emollient active ingredients such as Argan Oil, Rice Bran Oil, Kigelia Africana, and Quillaja Saponaria to help rebuild the protective skin barrier and maintain optimal hydration levels.
   - **Usage:** Apply morning and/or evening to perfectly cleansed skin and massage gently until completely absorbed.
   - **Price:** 42.00 euros for 50 ml

2. **Fluid Moisturizing Body Cream**
   - **Description:** This body cream contains bioactive Hyaluronic Acid and Marine Collagen to effectively hydrate and restructure the skin, providing softness and elasticity, which is beneficial for dry and rough skin.
   - **Usage:** Apply after a shower or bath and massage until completely absorbed, focusing on dry area

In [21]:
# test 3
query = "Can you suggest a Dorabruschi serum that helps with redness and sensitive skin?"
response = ask(query, df)
print(response)

I recommend the "Sensitive skin moisturizer" from Dorabruschi's product line for redness and sensitive skin. This cream is specifically designed for sensitive skin, providing soothing and protective properties. It contains Aloe Vera juice and emollient oils that offer maximum hydration and protection for sensitive and reddened skin. The cream is rapidly absorbed, leaving the skin supple and prepared for makeup application. 

Usage Instructions: Apply the "Sensitive skin moisturizer" morning and/or evening to perfectly cleansed skin. Gently massage until completely absorbed.

Price: 39.00 euros for 50 ml airless bottle

Unfortunately, there is no specific serum in the Dorabruschi product line that targets redness and sensitive skin.


Import the Q&A pairs spreadsheet and populate it with answers to the list of questions.

In [22]:
file_path = 'drive/MyDrive/colab_notebooks/independent_study/data/qa_pairs.xlsx'
qa = pd.read_excel(file_path)
qa.head()

,Question,Custom-GPT,RAG-GPT
0,I'm 25 and have combination skin. What Dorabru...,"For your combination skin, I recommend two cle...",NaN
1,What's the best Dorabruschi moisturizer for so...,Here's a suitable Dorabruschi moisturizing cre...,NaN
2,Can you suggest a Dorabruschi serum that helps...,NaN,NaN
3,Which Dorabruschi products are most effective ...,NaN,NaN
4,What Dorabruschi treatment would you recommend...,NaN,NaN


In [23]:
# function to generate a response for a list of queries
def answer_queries(queries: list[str]):
    responses = []
    for query in queries:
        response = ask(query, df)
        responses.append(response)
    return responses

In [29]:
# function to extract questions and add responses to Q&A df
def generate_responses(qa: pd.DataFrame, column: str):
    qa_new = qa
    for index, row in qa.iterrows():
        query = row['Question']
        response = ask(query, df)
        qa_new.loc[index, column] = response
    return qa_new

In [ ]:
qa_new = generate_responses(qa, 'RAG-GPT')

In [ ]:
qa_new[['Question', 'RAG-GPT']].head()

,Question,RAG-GPT
0,I'm 25 and have combination skin. What Dorabruschi cleanser should I use for daily skincare?,"For your daily skincare routine with combination skin, I recommend using the ""Delicate sebum-balancing cleansing base"" from Dorabruschi. This gentle washing base is derived from Coconut Oil and Glucose, providing a rebalancing, moisturizing, and soothing action. It is ideal for impure and reddened skin, suitable for all skin types, including combination skin.\n\n**Product Recommended:**\n- Pro..."
1,What's the best Dorabruschi moisturizer for someone living in a dry climate with dry skin?,"For someone living in a dry climate with dry skin, the best Dorabruschi moisturizer would be the ""Nourishing moisturizing cream for first wrinkles."" This 24-hour cream is designed to deeply nourish the skin and preserve its natural integrity, making it ideal for combating dryness. The concentration of emollient active ingredients such as Argan Oil, Rice Bran Oil, Kigelia Africana, and Quillaja..."
2,Can you suggest a Dorabruschi serum that helps with redness and sensitive skin?,"I recommend the ""Sensitive skin moisturizer"" from Dorabruschi for redness and sensitive skin. This cream is based on Aloe Vera juice and emollient oils, providing maximum hydration and protection for sensitive and reddened skin. It has soothing properties that help calm and protect delicate skin. The cream is rapidly absorbed, leaving the skin supple and prepared for makeup application. \n\n**..."
3,Which Dorabruschi products are most effective for deep wrinkles around the mouth?,"For deep wrinkles around the mouth, I recommend the following Dorabruschi products:\n\n1. **Anti-wrinkle cream K** - This cream is rich in emollient ingredients like cod liver oil and sweet almond oil, which help to soften and improve skin elasticity. It is specifically designed to target wrinkles and aging concerns. Apply this cream in the evening to perfectly cleansed skin for optimal result..."
4,What Dorabruschi treatment would you recommend for acne scars on oily skin?,"For acne scars on oily skin, I recommend the following Dorabruschi products:\n\n1. **Acne paste** (Price: 26.00 euros, Quantity: 30 ml):\n - **Description**: This paste is specifically designed for the treatment of pimples, blackheads, and enlarged pores. It contains ingredients like Zinc Oxide, Sulfur, and Salicylic Acid that help in purifying the skin and reducing imperfections.\n - **Us..."


In [ ]:
# test
print(qa_new['Question'][59], '\n\n', qa_new['RAG-GPT'][59])

Which Dorabruschi products help with oil control without drying out the skin? 

 To help with oil control without drying out the skin, I recommend the following Dorabruschi products:

1. **Delicate sebum-balancing cleansing base**
   - **Description:** This cleansing base is extremely gentle and rebalances the skin while providing moisture and soothing effects.
   - **Usage:** Apply a small amount to damp skin in the morning and evening, massage gently, and rinse thoroughly.
   - **Price:** 22.00 euros for 165 ml

2. **Rebalancing face cream**
   - **Description:** This light emulsion contains a purifying complex and sebum-regulating action, ideal for moisturizing impure, acne-prone skin without drying it out.
   - **Usage:** Apply morning and evening to cleansed skin and massage gently until absorbed.
   - **Price:** 39.00 euros for 50 ml

3. **Toning lotion**
   - **Description:** This non-alcoholic tonic lotion contains plant extracts known for their balancing properties, ideal for 

In [ ]:
qa_new.to_excel('qa_new.xlsx', index=False)

In [ ]:
files.download('qa_new.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

12 products are missing from the generated question-routine base. Questions have drafted to specifically include those products. Instruct the RAG model to produce a routine for each.

In [ ]:
file_path = 'drive/MyDrive/colab_notebooks/independent_study/data/missing_products.xlsx'
qa_missing = pd.read_excel(file_path)
qa_missing.head()

,Question,RAG-GPT
0,I'm looking for an effective all-in-one cleans...,NaN
1,I'm an athlete who often gets calluses from ru...,NaN
2,"My hands get very dry and cracked, especially ...",NaN
3,I’m noticing my skin losing elasticity due to ...,NaN
4,I'm in search of an anti-aging treatment that ...,NaN


In [ ]:
qa_missing_new = generate_responses(qa_missing)

In [ ]:
print(qa_missing_new.iloc[11, 1])

I recommend the "Roll-on anti-wrinkle eye contour fluid" from Dorabruschi for your concern about smoothing out expression lines around your eyes. This product is a highly concentrated solution in bioactive Hyaluronic Acid with 3 molecular weights and Saccharide Isomerate, which immediately smoothes expression lines around the eyes. Its roll-on applicator makes it easy to apply, even on the go, and it is quickly absorbed, making it ideal for those with little time. The tensor effect of this product is particularly welcome for special occasions.

**Product:** Roll-on anti-wrinkle eye contour fluid  
**Usage:** Apply with the roll-on on expression lines, focusing on the most evident signs. Allow it to absorb before applying makeup.  
**Price:** 39.00 euros for 10 ml  

This product is perfect for a quick and effective solution to address your concern about smoothing out expression lines around your eyes.


In [ ]:
qa_missing_new.to_excel('qa_missing_new.xlsx', index=False)

In [ ]:
files.download('qa_missing_new.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Evaluation
Generate responses to 25 evaluation questions drafted by Dorabruschi specialists.

In [30]:
file_path = 'drive/MyDrive/colab_notebooks/independent_study/data/model_eval.xlsx'
eval = pd.read_excel(file_path)
eval.head()

,Question,Custom GPT,RAG,Fine-tuning
0,"I have combination skin, which tends to be shi...",NaN,NaN,NaN
1,I have dry skin and would like a nourishing pr...,NaN,NaN,NaN
2,I am 65 years old and have several signs of ag...,NaN,NaN,NaN
3,"I can see my eye area aging, I notice many mor...",NaN,NaN,NaN
4,I turn 50 in a month and would like to arrive ...,NaN,NaN,NaN


In [31]:
eval_new = generate_responses(eval, 'RAG')

In [32]:
eval_new.loc[:5, ['Question', 'RAG']]

,Question,RAG
0,"I have combination skin, which tends to be shi...",I recommend the following beauty routine using...
1,I have dry skin and would like a nourishing pr...,"I recommend the ""FF toner"" from Dorabruschi fo..."
2,I am 65 years old and have several signs of ag...,"Based on your concerns of wrinkles, sagging sk..."
3,"I can see my eye area aging, I notice many mor...",For your concerns about aging in the eye area ...
4,I turn 50 in a month and would like to arrive ...,"For your upcoming milestone birthday party, I ..."
5,I would like a “shock” program to get back in ...,"For a ""shock"" program to treat cellulite and b..."


In [33]:
print(eval_new.loc[10, 'RAG'])

For your 25-year-old daughter with normal skin, here is a tailored beauty routine using products from Dorabruschi's product line:

1. **Gentle Cleansing Milk**: Start her routine with the **Gentle Cleansing Milk** to cleanse her face in the morning and evening. This product will effectively remove impurities without stripping the skin of its natural oils. Price: 23.00 euros for 200 ml. Usage: Apply directly to face and neck, then remove with warm water or a cotton pad.

2. **Nourishing Moisturizing Cream for First Wrinkles**: Follow up with the **Nourishing Moisturizing Cream for First Wrinkles** to deeply nourish her skin and maintain its optimal hydration levels. This cream will help preserve her skin's integrity and elasticity. Price: 42.00 euros for 50 ml. Usage: Apply morning and/or evening to perfectly cleansed skin.

3. **ACE 10% Multivitamin Concentrate**: Incorporate the **ACE 10% Multivitamin Concentrate** into her routine to provide her skin with a revitalizing cocktail of v

In [34]:
eval_new.to_excel('model_eval_rag.xlsx', index=False)
files.download('model_eval_rag.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>